In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [16]:
with open("api-token") as f:
    api_token = f.read().strip()

model_device = "cuda"
tokenizer_device = "cpu"
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16)
model.to(model_device)

tokenizer = AutoTokenizer.from_pretrained(model_id)

model.generation_config.max_length = 4096

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
from proj import dataloader
import pandas as pd

dataset = dataloader.load_billsum(
    "local_datasets"
)

merged_kw = {"test": pd.read_csv(".temp/keywords/billsum/test_merged_kw.csv"),
             "train": pd.read_csv(".temp/keywords/billsum/train_merged_kw.csv")
             }


In [25]:
message = ("<|begin_of_text|><|start_header_id|>system<|end_header_id|>"
           "Cutting Knowledge Date: December 2023\n"
           "Today Date: 31 Jul 2024\n"
           "You modify given summary using context from the text to contain information related to keywords from a given comma-separated list of keywords\n"
           "<|eot_id|><|start_header_id|>user<|end_header_id|>"
           "text: " + dataset["test"]["text"][0] + "\n"
           "summary: " + dataset["test"]["summary"][0] + "\n"
           "comma-separated list of keywords: monetary"
           "<|eot_id|>"
           "<|start_header_id|>assistant<|end_header_id|>")

In [26]:
inputs = tokenizer(message, return_tensors="pt").to(model_device)
outputs = model.generate(**inputs, do_sample=False)
response = tokenizer.batch_decode(outputs)[0]

print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>Cutting Knowledge Date: December 2023
Today Date: 31 Jul 2024
You modify given summary using context from the text to contain information related to keywords from a given comma-separated list of keywords
<|eot_id|><|start_header_id|>user<|end_header_id|>text: SECTION 1. ENVIRONMENTAL INFRASTRUCTURE.

    (a) Jackson County, Mississippi.--Section 219 of the Water 
Resources Development Act of 1992 (106 Stat. 4835; 110 Stat. 3757) is 
amended--
        (1) in subsection (c), by striking paragraph (5) and inserting 
    the following:
        ``(5) Jackson county, mississippi.--Provision of an alternative 
    water supply and a project for the elimination or control of 
    combined sewer overflows for Jackson County, Mississippi.''; and
        (2) in subsection (e)(1), by striking ``$10,000,000'' and 
    inserting ``$20,000,000''.
    (b) Manchester, New Hampshire.--Section 219(e)(3) of the Water 
Resources De